In [1]:
import pandas as pd
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt
import math
plt.style.use('ggplot')
%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


In [2]:
#MAPPING AV SEKTORER

KOMMUNIKASJON = ["ADE.OL","SCHA.OL","SCHB.OL","TEL.OL"]
INDUSTRI = ["AFG.OL","AMSC.OL","BON.OL","FJORD.OL","GOGL.OL","HEX.OL","KOG.OL","MPCC.OL","NEL.OL","NAS.OL","SNI.OL","TOM.OL","VEI.OL","WALWIL.OL","WWI.OL"]
FINANS = ["AKER.OL","AXA.OL","B2H.OL","DNB.OL","GJF.OL","NOFI.OL","SRBANK.OL","STB.OL"]
ENERGI = ["AKERBP.OL","AKSO.OL","BDRILL.OL","BWLPG.OL","BWO.OL","DNO.OL","EQNR.OL","FRO.OL","PGS.OL","SUBC.OL","TGS.OL"]
IT = ["ASETEK.OL","ATEA.OL","IDEX.OL","KIT.OL","NOD.OL","TIETOO.OL"]
KONSUMVARER = ["AUSS.OL","BAKKA.OL","GSF.OL","LSG.OL","MOWI.OL","ORK.OL","SALM.OL"]
HELSEVERN = ["BGBIO.OL","NANO.OL","PCIB.OL","PHO.OL"]
MATERIALER = ["ELK.OL","NHY.OL","YAR.OL"]
EINDOM = ["ENTRA.OL","OLT.OL"]
FORBRUKSVARER = ["EPR.OL","GIG.OL","KOA.OL","XXL.OL"]
FORSYNING = ["FKRAFT.OL","SSO.OL"]

#FULL LISTE
OSEBX = KOMMUNIKASJON+INDUSTRI+FINANS+ENERGI+IT+KONSUMVARER+HELSEVERN+MATERIALER+EINDOM+FORBRUKSVARER+FORSYNING



In [5]:
 def Create_Dataframe(list_of_tickers,number_of_firms,sector):
    """
    Returns a datafram with returns,volatility and sharpe ratio. The DF is sorted by SR.
    List_of_tickers = list: format of tickers that should be analyzed
    Number_of_firms = int: the number of top firms you want to be returned
    Sector = str: The name of the sector you are analyzing. 
    """
    #Download the data
    df_prices = yf.download(tickers=list_of_tickers,start="2017-03-05",end="2021-03-05",interval="1d",group_by="ticker")

    #Clean the data
    df_prices.drop(labels=["Open","High","Low","Close","Volume"],axis=1,inplace=True,level=1)
    df_prices = df_prices.droplevel(level=1,axis=1)

    #Calculate the returns for each ticker
    df_returns = df_prices.pct_change(1).dropna()

    #Perform analysis
    #Store the return,vol,SR
    return_dict = {}
    volatility_dict = {}
    sharpe_dict = {}

    for x in list_of_tickers:
        return_dict[x]= round(((((1+df_returns[x].mean())**252)-1)*100),2)

    for x in list_of_tickers:
        volatility_dict[x]=round((df_returns[x].std()*math.sqrt(252)*100),2)

    for (k,return_value),(k2,volatility_value) in zip(return_dict.items(),volatility_dict.items()):
        sharpe_dict[k] = round((return_value/volatility_value),2)

    #Sort the data to get the top sharpe
    top_sharpe = sorted(sharpe_dict.items(), key=lambda x: x[1],reverse=True)[0:number_of_firms]
    #Create an ordered list of just the tickers in top sharpe
    top_sharpe_list = [x for x,y in top_sharpe]
    
    #Get lists with the values
    top_sharpe_list_ret = [return_dict[x] for x in top_sharpe_list]
    top_sharpe_list_vol = [volatility_dict[x] for x in top_sharpe_list]
    top_sharpe_list_sr = [sharpe_dict[x] for x in top_sharpe_list]
    
    #Create the dataframe
    sector_list = [sector]
    idx=pd.MultiIndex.from_product([sector_list,top_sharpe_list],names=["Sector","Ticker"])
    df = pd.DataFrame(index=idx,columns=["Return","Volatility","Sharpe Ratio","Revenue Growth",
                                         "Cost Growth","EBIT Growth","Income Growth","Beta","EPS","P to B"])
    df["Return"]=top_sharpe_list_ret
    df["Volatility"]=top_sharpe_list_vol
    df["Sharpe Ratio"]=top_sharpe_list_sr
    
    #######################################
    ######## Financial analysis ###########
    #######################################
    
    revenue_data = []
    cost_data = []
    EBIT_data = []
    Inc_data = []
    beta_data = []
    EPS_data = []
    P_t_B_data = []
    
    #Perform analysis
    for x in top_sharpe_list:
        #Get data
        stock_object = yf.Ticker(x)
        revenue_growth =0
        cost_growth=0
        ebit_growth=0
        income_growth=0
        beta_factor=0
        EPS_factor=0
        P_t_B_factor=0
        
        try:
            revenue_growth = stock_object.financials.loc["Total Revenue"].pct_change(-1).mean()
            revenue_data.append(revenue_growth)
        except IndexError:
            revenue_data.append(revenue_growth)
            
        try:
            cost_growth = stock_object.financials.loc["Total Operating Expenses"].pct_change(-1).mean()
            cost_data.append(cost_growth)
        except IndexError or HTTPError:
            cost_data.append(cost_growth)
        
        try:
            ebit_growth = stock_object.financials.loc["Ebit"].pct_change(-1).mean()
            EBIT_data.append(ebit_growth)
        except IndexError or HTTPError:
            EBIT_data.append(ebit_growth)
            
        try:
            income_growth = stock_object.financials.loc["Net Income"].pct_change(-1).mean()
            Inc_data.append(income_growth)
        except IndexError or HTTPError:
            Inc_data.append(income_growth)
            
        try:
            beta_factor = stock_object.info["beta"]
            beta_data.append(beta_factor)
        except IndexError or HTTPError:
            beta_data.append(beta_factor)
        
        try:
            EPS_factor = stock_object.info["trailingEps"]
            EPS_data.append(EPS_factor)
        except IndexError or HTTPError:
            EPS_data.append(EPS_factor)
            
        try:
            P_t_B_factor = stock_object.info["priceToBook"]
            P_t_B_data.append(P_t_B_factor)
        except IndexError or HTTPError:
            P_t_B_data.append(P_t_B_factor)
            
    #Add all the data to the dataframe
    df["Revenue Growth"]=revenue_data
    df["Cost Growth"]=cost_data
    df["EBIT Growth"]=EBIT_data
    df["Income Growth"]=Inc_data
    df["Beta"]=beta_data
    df["EPS"]=EPS_data
    df["P to B"]=P_t_B_data
    
    return df    

In [6]:
df_kommunikasjon = Create_Dataframe(KOMMUNIKASJON,10,"Koummunikasjon")
df_industri = Create_Dataframe(INDUSTRI,10,"Industri")
df_finans = Create_Dataframe(FINANS,10,"Finans")
df_energi = Create_Dataframe(ENERGI,10,"Energi")
df_it = Create_Dataframe(IT,10,"IT")
df_konsumvarer = Create_Dataframe(KONSUMVARER,10,"Konsumvarer")
df_helsevern = Create_Dataframe(HELSEVERN,10,"Helsevern")
df_materialer = Create_Dataframe(MATERIALER,10,"Materialer")
df_eindom = Create_Dataframe(EINDOM,10,"Eindom")
df_forbruksvarer = Create_Dataframe(FORBRUKSVARER,10,"Forbruksvarer")
df_forsyning = Create_Dataframe(FORSYNING,10,"Forsyning")

[*********************100%***********************]  4 of 4 completed


ValueError: If using all scalar values, you must pass an index

In [6]:
list_of_dfs = [df_kommunikasjon,df_industri,df_finans,df_energi,df_it,df_konsumvarer,
               df_helsevern,df_materialer,df_eindom,df_forbruksvarer,df_forsyning]

result = pd.concat(list_of_dfs)

In [8]:
result.to_excel(r"C:\Users\marius\Desktop\Marius stuff\Koding\ferdig.xlsx")

In [7]:
result

Return  Volatility  Sharpe Ratio  Revenue Growth  \
Sector         Ticker                                                        
Koummunikasjon ADE.OL      34.06       30.15          1.13        0.000000   
               SCHB.OL     18.22       24.99          0.73        0.063606   
               SCHA.OL     15.59       23.72          0.66        0.063606   
               TEL.OL       2.42       19.71          0.12       -0.030522   
Industri       NEL.OL     133.84       57.36          2.33        0.862231   
               TOM.OL      65.69       31.97          2.05        0.122759   
               BON.OL      55.77       30.36          1.84       -0.086932   
               KOG.OL      26.39       29.62          0.89        0.193821   
               FJORD.OL    19.28       27.24          0.71        0.058621   
               AMSC.OL     16.58       26.95          0.62        0.000000   
               AFG.OL      12.15       20.32          0.60        0.244635   
               HEX.OL      20.35       39.62          0.51        0.502493   
               VEI.OL      10.90       24.78          0.44        0.086306   
               GOGL.OL      6.38       44.83          0.14        0.000000   
Finans         GJF.OL      24.32       18.39          1.32        0.025652   
               SRBANK.OL   20.92       19.99          1.05        0.147120   
               DNB.OL      15.33       20.23          0.76        0.067709   
               AKER.OL     20.96       28.20          0.74       -0.094717   
               STB.OL      14.21       22.11          0.64        0.381315   
               NOFI.OL     16.27       33.56          0.48        0.228430   
               AXA.OL      -9.51       38.35         -0.25        1.034337   
               B2H.OL     -14.03       38.87         -0.36        0.239322   
Energi         AKERBP.OL   43.29       34.32          1.26        0.000000   
               BWO.OL      60.21       51.64          1.17        0.000000   
               BWLPG.OL    51.52       48.46          1.06        0.199407   
               FRO.OL      44.85       48.09          0.93        0.000000   
               TGS.OL      29.18       35.04          0.83        0.000000   
               EQNR.OL     11.94       22.50          0.53        0.000000   
               PGS.OL      31.40       65.44          0.48        0.068140   
               SUBC.OL      1.88       32.16          0.06        0.000000   
               DNO.OL      -4.62       43.70         -0.11        0.000000   
               AKSO.OL    -24.58       38.55         -0.64        0.053995   
IT             KIT.OL     277.09       36.09          7.68        0.166270   
               ASETEK.OL  577.23       84.64          6.82        0.000000   
               NOD.OL     163.83       43.43          3.77        0.135425   
               IDEX.OL    172.84       87.22          1.98       -0.434312   
               TIETOO.OL   10.90       58.86          0.19        0.000000   
               ATEA.OL    -49.40       27.39         -1.80        0.055385   
Konsumvarer    SALM.OL     43.98       31.40          1.40        0.109584   
               BAKKA.OL    40.29       29.35          1.37        0.146566   
               MOWI.OL     22.81       23.81          0.96        0.000000   
               GSF.OL      36.41       38.13          0.95        0.080301   
               LSG.OL      19.61       30.17          0.65        0.057772   
               ORK.OL      10.38       18.14          0.57        0.049344   
               AUSS.OL     16.12       31.96          0.50        0.073222   
Helsevern      PHO.OL      92.19       59.37          1.55        0.268221   
               PCIB.OL     63.73       65.82          0.97        0.000000   
               BGBIO.OL     8.48       59.16          0.14        0.000000   
               NANO.OL    -12.75       69.02         -0.18       -0.519108   
Materialer     YAR.OL      19.62       20.57    